### IEMS 308 - Question Answer System (w/o fluff and errors)

This codebase seeks to be easily maintainable, able to be run from the jupyter notebook stack.

In [4]:
%%time
import re
import requests
import subprocess
import glob
import time
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from elasticsearch import Elasticsearch
from nltk.corpus import stopwords

subprocess.Popen('/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/elasticsearch-7.6.1/bin/elasticsearch')
time.sleep(5)
res = requests.get('http://localhost:9200')
es = Elasticsearch()

path = '/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/data/*.txt'
files = glob.glob(path)
corpus = []
for document in files:
    with open(document, 'r', errors='ignore') as single_document:
        read_document = single_document.read().replace('\n', ' ')
    corpus = corpus + [read_document]    
    
#Iterate over documents in corpus and index them
i = 1
for document in tqdm(corpus):
    es.index(index='docs',doc_type='article', id=i, body={'document' : document}) 
    i = i + 1       

corpus_sent = []
for doc in tqdm(corpus):
    document_sentences = sent_tokenize(doc)
    corpus_sent.append(document_sentences)

100%|██████████| 730/730 [00:24<00:00, 29.59it/s]

CPU times: user 26.6 s, sys: 688 ms, total: 27.3 s
Wall time: 56.9 s


Usually takes about 1 minute to run startup.

In [ ]:
question = input('Type your question and hit enter: \n\n')

while ('Stop' not in question):

    qtokens = word_tokenize(question)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['?','Which','went','What','of','or','is','with','this','Who','the'])

    keywords = []
    for word in qtokens:
        if word not in stop_words:
            keywords += [word]

    #determining question type
    if 'CEO' in keywords:
        question_type = 1
    elif 'bankrupt' in keywords:
        question_type = 2
    else:
        question_type = 3
    #CEO questions
    if question_type == 1:
        keywords_str = ''
        for word in keywords:
            keywords_str += ' ' + word
        search_term = keywords + [keywords_str]  
        
        doc_catcher = es.search(index='docs',q=search_term) #find docs with matches

        doc_matchid = [] #store top match ids
        for hit in tqdm(range(0,min(doc_catcher['hits']['total']['value'],1))):
            doc_matchid += [doc_catcher['hits']['hits'][hit]['_id']]

        i = 1
        sentence_list = []
        for id_num in doc_matchid: #for each document   
            document = corpus_sent[int(id_num)-1]
            for sentence in tqdm(document): #for each sentence in each document
                es.index(index='sentences',doc_type='sentence', id=i, body={'sentence' : sentence})
                i += 1

        query_sentences = es.search(index='sentences',q=search_term,size=10)

        sent_matchid = [] #store sentence matches
        for hit in tqdm(range(0,min(query_sentences['hits']['total']['value'],10))):
            sent_matchid += [query_sentences['hits']['hits'][hit]['_id']]

        regex = '(?<='+keywords[1]+'\sCEO\s)[A-Z][a-z]+\s[A-Z][a-z]+'
        for sent_id in sent_matchid:
            sentence = es.get(index='sentences',doc_type='sentence',id=int(sent_id))['_source']['sentence']
            try:
                name = re.search(regex, sentence).group()
                break
            except: pass
        print(name) #return the CEO name
    
    #Bankrupcy questions
    elif question_type == 2:
        keywords_str = ''
        for word in keywords:
            keywords_str += ' ' + word
        search_term = keywords + [keywords_str]  

        doc_catcher = es.search(index='docs',q=search_term)

        doc_matchid = []
        for hit in range(0,min(doc_catcher['hits']['total']['value'],3)):
            doc_matchid += [doc_catcher['hits']['hits'][hit]['_id']]

        i = 1
        sentence_list = []
        for id_num in doc_matchid:   
            document = corpus_sent[int(id_num)-1]
            for sentence in document:
                es.index(index='sentences',doc_type='sentence', id=i, body={'sentence' : sentence})
                i += 1

        query_sentences = es.search(index='sentences',q=search_term,size=50)

        sent_matchid = []
        for hit in tqdm(range(0,min(query_sentences['hits']['total']['value'],50))):
            sent_matchid += [query_sentences['hits']['hits'][hit]['_id']]

        relevant_sentences = [] #store most relevant sentences
        for sent_id in sent_matchid:
            sentence = es.get(index='sentences',doc_type='sentence',id=int(sent_id))['_source']['sentence']
            if 'bankrupt' in sentence:           
                relevant_sentences.append(sentence)

        relevant_subsentences = [] #clean up the sentences to return smaller chunks
        for sentence in relevant_sentences:
            if 'France' not in sentence and 'Europe' not in sentence: #france/europe keeps going bankrupt so remove this
                bankrupt_span = re.search('bankrupt',sentence).span()
                relevant_subsentences.append(sentence[bankrupt_span[0]-20:bankrupt_span[1]+20])

        #extract companies
        regex = '(?<!\.\s)\s[A-Z][a-z]+(\s[A-Z][a-z]+)?'
        answer_list = []
        for subsentence in relevant_subsentences:
            try:
                bankrupt_company = re.search(regex, subsentence).group().strip()
                answer_list.append(bankrupt_company)
            except: pass
        print(answer_list) #return companies
        

    #GDP affectation question
    else: 
        search_term1 = 'gross domestic product shrank'
        search_term2 = '(GDP | (gross & domestic & product)) & (increase|decrease|rose|fell|up|down|change)'
        search_terms = [search_term1, search_term2]  

        answers = []
        second_halves = []
        for search_term in search_terms:
            first_finished = 0
            doc_catcher = es.search(index='docs',q=search_term)

            doc_matchid = []
            for hit in tqdm(range(0,min(doc_catcher['hits']['total']['value'],3))):
                doc_matchid += [doc_catcher['hits']['hits'][hit]['_id']]

            i = 1
            sentence_list = []
            for id_num in doc_matchid:   
                document = corpus_sent[int(id_num)-1]
                for sentence in tqdm(document):
                    es.index(index='sentences',doc_type='sentence', id=i, body={'sentence' : sentence})
                    i += 1

            query_sentences = es.search(index='sentences',q=search_term,size=5)

            sent_matchid = []
            for hit in range(0,min(query_sentences['hits']['total']['value'],3)):
                sent_matchid += [query_sentences['hits']['hits'][hit]['_id']]               
            
            do_not_enter = 0
            for sent_id in sent_matchid:
               
                sentence = es.get(index='sentences',doc_type='sentence',id=int(sent_id))['_source']['sentence']
                stokens = word_tokenize(sentence)
                stokens_pos = nltk.pos_tag(stokens)  #POS tagging
                
                split_point = 0
                percent_present = 0
                #detect percentages
                for percent_search in stokens_pos:
                    if percent_search[0] in ['percent',"%"]:
                        percent_present = 1
                        break
                    split_point += 1    
                if percent_present == 1: #if present, split
                    stokens_pos_subset = stokens_pos[0:split_point]
                    stokens_pos_subset2 = stokens_pos[split_point-1:]

                    for percent_search in reversed(stokens_pos_subset):
                        if percent_search[1] == 'NN' and do_not_enter==0 and first_finished==0: #nn = noun POS tag
                            answer = percent_search[0]
                            do_not_enter = 1
                            first_finished = 1
                            second_halves += [stokens_pos_subset2]
                            break
                        elif percent_search[1] == 'NNP': #nnp = proper noun POS tag
                           try: 
                               answer = re.search('[A-Z][A-Z]+',percent_search[0].replace(u"\u2122", '')).group()
                               second_halves += [stokens_pos_subset2]
                               break
                           except: pass
                    answers += [answer]  
        print("GDP is affected by:")
        print(list(set(answers)))

        #second question
        followup = input("Enter property from above list to examine its percentage influence on GDP. \n\n")

        #figuring out where to search, hardcoded some typical results to speed up
        if 'PMI' in followup:
            percent_half = second_halves[1]
        elif 'unemployment' in followup:
            percent_half = second_halves[0]
        else:
            percent_half = second_halves[2]

        percent_half_string = ''
        for percent_search in percent_half:
            percent_half_string += (' ' + percent_search[0])

        percent_regex = '\s([0-9]+|[a-zA-Z]+-?[a-zA-Z]*|[0-9]+\.[0-9]+)\s?(%|percent)(age point)?'
        list_percents = re.findall(percent_regex, percent_half_string)

        percent_as_tuple = list_percents[-1]

        percent_string = ''
        for term in percent_as_tuple:
            percent_string += (' ' + term)
        percent_string = percent_string.strip()
        
        print(percent_string) 
        
    es.indices.delete(index="sentences",ignore=[400, 404])
    question = input('Type question and hit return: \n\n')

Type your question and hit enter: 

 Which companies went bankrupt in September of 2005?


100%|██████████| 50/50 [00:00<00:00, 190131.64it/s]


['Estar']


### Runtimes

- CEO questions: takes about 1 minute to run.
- Bankrupcy questions: takes about 3 minutes to run.
- GDP questions: grab a coffee, this takes at least 6 minutes to run, but the subsequent followup is instantaneous. 